<a href="https://colab.research.google.com/github/camilotayac/computational_chemistry-TC/blob/master/Colab/Computational_Electrophysiology_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧙‍♂️ **Computational Electrophysiology** Tutorial ♾️ Tutorial de **Computational Electrophysiology**

🐉 **@Tyac**

🏦 **Universidad de Caldas**

🌇 **Manizales - Colombia**

In [ ]:
# @title # 0️⃣👻 **Recursos necesarios para el proceso y si quiere visualizar estructuras**
# @markdown ### **🇺🇸** - Type the version of **GROMACS** to be installed
# @markdown ### **🇪🇸** - Escriba la versión a instalar de **GROMACS**
gromacs = "2024.2" # @param {"type":"string"}
# @markdown ### **🇺🇸** - Select whether you are installing **GROMACS** with **GPU** or not.
# @markdown ### **🇪🇸** - Seleccione si va a instalar **GROMACS** con **GPU** o no
gpu = "False" # @param ["True", "False"] {allow-input: true}
!pip install -q condacolab
import condacolab
condacolab.install()
if gpu=='True':
  !conda install gromacs=$gromacs=nompi_cuda*
else:
  !conda install gromacs=$gromacs
!gmx -version
def leer_archivo(ruta_archivo):
    try:
        with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
            contenido = archivo.read()
        return contenido
    except FileNotFoundError:
        return f"Error: El archivo '{ruta_archivo}' no se encontró."
    except Exception as e:
        return f"Error al leer el archivo: {str(e)}"

!pip install py3Dmol

In [ ]:
# @title ## 📁 Files for analysis ♾️ Archivos para el análisis
# @markdown ### **🇺🇸** - Select whether you will work with tutorial files or with your own files
# @markdown ### **🇪🇸** - Seleccione si va a trabajar con archivos del tutorial o con archivos propios
files = 'Tutorial files - Archivos del tutorial'# @param ['Tutorial files - Archivos del tutorial', 'Own files - Archivos propios']
# @markdown #### 🔴 **🇺🇸 - Note:** add the address of your folder, example: ***folder_a*** or ***folder_a/folder_b***
# @markdown #### 🔴 **🇪🇸 - Nota:** agregue la dirección de su carpeta, ejemplo: ***carpeta_a*** o ***carpeta_a/carpeta_b***
drive_folder_path = "MD" #@param {type:"string"}

if files == 'Tutorial files - Archivos del tutorial':
  %cd /content
  !git clone https://github.com/camilotayac/computational_chemistry-TC.git
  %cd computational_chemistry-TC/Examples/CG
elif files == 'Own files - Archivos propios':
  from google.colab import drive
  drive.mount('/content/drive')
  carpeta = "/content/drive/MyDrive/" + drive_folder_path
  %cd {carpeta}

/content
fatal: destination path 'computational_chemistry-TC' already exists and is not an empty directory.
/content/computational_chemistry-TC/Examples/CG


In [ ]:
# @title 1️⃣Visualización del sistema de membrana CG
import py3Dmol

def visualizar_estructura_gro(archivo_gro, estilo='stick', seleccionar='todo', radio=0.5):
    """
    Visualiza una estructura molecular a partir de un archivo .gro utilizando py3Dmol.

    Parámetros:
        archivo_gro (str): Ruta del archivo .gro.
        estilo (str): Estilo de visualización ('stick', 'sphere', 'line', 'cartoon').
        seleccionar (str): Parte de la estructura a visualizar ('todo', 'lipidos', 'proteina', 'todo_menos_agua').
        radio (float): Radio de visualización para los estilos 'sphere' o 'stick'.
    """
    try:
        # Lee el contenido del archivo .gro
        with open(archivo_gro, 'r') as f:
            estructura = f.read()
    except FileNotFoundError:
        print(f"Error: El archivo '{archivo_gro}' no se encontró.")
        return
    except Exception as e:
        print(f"Error al leer el archivo: {e}")
        return

    # Inicializa el visor 3D
    vista = py3Dmol.view(width=1200, height=600)
    vista.addModel(estructura, 'gro')  # Carga el archivo como modelo

    # Verifica si el estilo proporcionado es válido
    estilos_permitidos = ['stick', 'sphere', 'line', 'cartoon']
    if estilo not in estilos_permitidos:
        print(f"Estilo '{estilo}' no reconocido. Usando 'stick' por defecto.")
        estilo = 'stick'

    # Define los parámetros del estilo
    estilo_parametros = {estilo: {}}
    if estilo in ['stick', 'sphere']:
        estilo_parametros[estilo]['radius'] = radio

    # Filtra la parte de la estructura a visualizar
    if seleccionar == 'lipidos':
        vista.setStyle({'resn': 'DPP'}, estilo_parametros)  # Selecciona los lípidos (DPP es un residuo común para lípidos en Martini)
    elif seleccionar == 'proteina':
        vista.setStyle({'resn': 'PROA'}, estilo_parametros)  # Selecciona la proteína (PROA es un residuo común para proteínas en Martini)
    elif seleccionar == 'todo_menos_agua':
        vista.setStyle({'not': {'resn': 'W'}}, estilo_parametros)  # Muestra todo excepto el agua (asumiendo que 'W' es el residuo para agua)
    else:
        vista.setStyle(estilo_parametros)  # Muestra todo el sistema

    # Ajusta el zoom para ver la selección
    vista.zoomTo()
    vista.show()
visualizar_estructura_gro('step6.6_equilibration.gro', estilo='sphere', seleccionar='todo_menos_agua',radio=1.2)

In [ ]:
# @title 2️⃣ Creando el sistema de Computational Electrophysiology
eje_x = 1 # @param {type:"slider", min:1, max:10, step:1}
eje_y = 1 # @param {type:"slider", min:1, max:10, step:1}
eje_z = 2 # @param {type:"slider", min:1, max:10, step:1}



!gmx genconf -f step6.6_equilibration.gro -nbox $eje_x $eje_y $eje_z -o step6.6_equilibration_2.gro

In [ ]:
# @title 3️⃣ Visualización del sistema de Computational Electrophysiology
visualizar_estructura_gro('step6.6_equilibration_2.gro', estilo='sphere', seleccionar='todo_menos_agua',radio=1.2)

In [ ]:
# @title 4️⃣ Creando new index
!echo -e 'a BB\na NA\na CL\nq'|gmx make_ndx -f step6.6_equilibration_2.gro -o index_1.ndx

In [ ]:
# @title 5️⃣ Ejecutando la Dinámica Molécular

!gmx grompp -f step7_production_2.mdp -o step7_production_2.tpr -c step6.6_equilibration_2.gro -p system.top -n index_2.ndx
!gmx mdrun -v -deffnm step7_production_2

In [ ]:
# @title 6️⃣ Visualización de la Dinámica Molecular
!echo -e '0'|gmx trjconv -s step7_production.tpr -f step7_production.xtc -o step7_production.pdb
molecules = []
with open("step7_production.pdb", "r") as infile:
    current_model = []
    for line in infile:
        if line.startswith("MODEL"):
            model_number = int(line.split()[1])
        if line.startswith("ENDMDL"):
            if model_number % 10 == 0:  # Extraer cada 10mo modelo
                molecules.append(current_model)
            current_model = []
        else:
            current_model.append(line)

# Crear una lista para almacenar los modelos PDB
molecules = []

# Cargar los modelos desde el archivo PDB principal
with open("step7_production.pdb", "r") as ifile:
    content = ifile.read().split("ENDMDL")  # Separar cada modelo por 'ENDMDL'

# Agregar cada modelo a la lista de moléculas
for model_content in content[:-1]:  # Ignorar último fragmento vacío
    molecules.append(model_content.strip())

# Inicializar el visor 3D
view = py3Dmol.view(width=800, height=600)

# Concatenar los modelos en un string compatible con Py3Dmol
models = ""
for i, mol in enumerate(molecules):
    models += f"MODEL {i + 1}\n{mol}\nENDMDL\n"

# Añadir los modelos al visor
view.addModelsAsFrames(models)

# Aplicar el estilo, excluyendo residuos con nombre 'W'
view.setStyle({'not': {'resn': 'W'}}, {"sphere": {'scale': 1.0}})
# Ajustar la vista y configurar la animación
view.zoomTo()
view.animate({'loop': "forward"})
view.show()


In [ ]:
contenido = leer_archivo('step6.0_minimization.mdp')
print(contenido)